# BLEU: Method forAutomatic Evaluation of Machine Translation

## Fabrication de l'hypothèse

Tout d'abod on télécharge un corpus anglais-français et on prendra 200 lignes de ce corpus. Nous utiliserons le corpus anglais en tant que source et le français en tant que corpus théorique. Nous ferons donc une hypothèse de traduction avec Google Traduction.

In [1]:
import numpy as np

fileSource = open("europarl-v7.fr-en.en", "r")
fileTarget = open("europarl-v7.fr-en.fr", "r")
fileHypo = open("hypothese-GoogleTrad-fr", "r")

# Fonction qui supprime la ponctuation d'un corpus
def suppPonct( uneLigne ):
    chars = [",",";",".","?","!",":",]
    for char in chars:
        uneLigne = uneLigne.replace(char,"")
    return uneLigne

N = 200
# On prends les 200 premieres phrases du texte source et de référence
source = [next(fileSource) for x in range(N)]
reference = [next(fileTarget) for x in range(N)]
# Mon fichier de traduction contient 200 lignes maximum
hypoGoogle = [next(fileHypo) for x in range(N)]

## Calcul précision d'un nGram

In [2]:
# Retourne la liste des nGrams en fonction d'un n et d'une phrase
def getList_nGram( n, unePhrase ):
    #Remove \n at the end
    unePhrase = suppPonct(unePhrase).split(" ")
    unePhrase[-1] = unePhrase[-1].rstrip()
    liste = []
    
    for i in range(0, (len(unePhrase)-n+1)):
        ngram = []
        for y in range(0,n):
            ngram.append(unePhrase[i+y])
            
        liste.append(ngram)
        
    return liste

In [3]:
# Return the global accuracy of the test in with nGram
def getAccuracy_nGram( n ):
    
    # Accuracy list of sentences
    acc = []
    for y in range(0,N):
        
        tempHypo = getList_nGram( n, hypoGoogle[y])
        # Check if the list is not empty
        # A list can be empty if the sentence have less than n word(s)
        if not not tempHypo:
            tempRef = getList_nGram( n, reference[y])        

            good = 0
            for i in range(0,len(tempHypo)):
                if tempHypo[i] in tempRef:
                    good += 1
            acc.append(good/len(tempHypo))
    
    return np.mean(acc)

## Calcul du score BLEU en fonction des nGrams

In [4]:
# Return the bleu score
def getBleuScore():
    nGramAccuracy = []
    # Calculate the accuracies from 1Grams to 4Grams
    for i in range(1,5):
        nGramAccuracy.append(getAccuracy_nGram(i))
        
    #print( "Probabilités des nGrams:", nGramAccuracy, "\n" )
        
    # Raise product of nGrams to power 1/4 give the BLEU score
    return np.power( np.prod(nGramAccuracy), 1/4 )

In [5]:
print( "Score BLEU Google Traduction sur 200 lignes Anglais -> Français: ", round(getBleuScore()*100, 2), "%" )

Score BLEU Google Traduction sur 200 lignes Anglais -> Français:  32.28 %
